In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os

In [ ]:
def main():
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    if face_cascade.empty():
        print("Error: Haar cascade not loaded.")
        return

    model = load_model('ams_tl.keras')

    data_dir = "C:/Users/adnan/OneDrive/Documents/notebook/dataset/cvpr_img_dataset"
    try:
        class_names = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))])
        print(f"Loaded {len(class_names)} classes.")
    except Exception as e:
        print(f"Error loading class names from directory: {e}")
        return

    cap = cv2.VideoCapture(0)
    cv2.namedWindow("attendance", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("attendance", 720, 500)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret: break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h, x:x+w]
            face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            face_img = cv2.resize(face_roi, (32, 32)) 
            face_img = face_img.astype("float32") / 255.0
            face_img = np.expand_dims(face_img, axis=0)


            prediction = model.predict(face_img, verbose=0)
            class_id = np.argmax(prediction)
            conf = np.max(prediction)

            if class_id < len(class_names):
                student_name = class_names[class_id]
            else:
                student_name = "Unknown"

            if conf < 0.80: 
                label = "Unknown"
            else:
                label = f"{student_name} ({conf*100:.1f}%)"

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        cv2.imshow("attendance", frame)
        if cv2.waitKey(1) & 0xFF == 27: 
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded 86 classes.
